# Тема семинара: отбор признаков

- Фильтрационные методы
- Оберточные методы
- Встроенные методы
- Метод главных компонент или PCA

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('Pokemon.csv')

In [ ]:
data

Columns description (it's crucial!)


- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round

In [ ]:
# fillna and drop useless cols

display(data.isnull().sum())
data['Type 2'] = data['Type 2'].fillna('No 2nd type')

data.drop(columns=['#', 'Name'], inplace=True)

In [ ]:
X = data.drop(columns='Legendary')
y = data['Legendary'].astype('int')

In [ ]:
y.value_counts(normalize=True)

# Make some default pipeline

In [ ]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

In [ ]:
# define cat_cols

cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [ ]:
cv_res1 = cross_validate(default_pipeline,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

In [ ]:
cv_res1

In [ ]:
cv_res1['test_score'].mean()

# Make pipeline more complicated

In [ ]:
# difficult pipeline

pipe_dif = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [ ]:
cv_res2 = cross_validate(pipe_dif,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

cv_res2

In [ ]:
cv_res2['test_score'].mean()

train_score - просто класс ! модель получилась сложная, только очевидно переобученная ...

согласны, узнали ?


# Introduce feature selectors

In [ ]:
data_tr = pipe_dif[:-1]

In [ ]:
data_tr

In [ ]:
X_tr = data_tr.fit_transform(X, y)
print(f'data shape after transformation is {X_tr.shape}')

1k признаков - многовато, добавим в пайплайн селектор

## Фильтрационные методы

Суть таких методов в том, чтобы для каждого признака посчитать некоторую метрику "связи" с целевым признаком. И в результате оставить топ-K признаков согласно выбранной метрике.

В том числе на лекции обсуждались:

 - статистика хи-квадрат
 - метрика mutual information

In [ ]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [ ]:
k_best = 30

pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectKBest(score_func=mutual_info_classif, k=k_best)), 
    ('model_', SVC(kernel='linear'))]
)



In [ ]:
cv_res = cross_validate(pipe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res

In [ ]:
# k best нужно подбирать

cv_res['test_score'].mean()

## Жадный метод отбора

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [ ]:
rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)

In [ ]:
X_tr.shape

In [ ]:
res = rfe.fit_transform(X_tr, y)
display(res.shape)
res

In [ ]:
pipe_rfe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', RFE(LogisticRegression(max_iter=1000),
                      n_features_to_select=30,
                      step=30
                     )), 
    ('model_', SVC(kernel='linear'))])

In [ ]:
cv_res3 = cross_validate(pipe_rfe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res3

In [ ]:
cv_res3['test_score'].mean()

## С помощью L1 регуляризации

In [ ]:
from sklearn.feature_selection import SelectFromModel

In [ ]:
sel = SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), threshold=1e-5)

In [ ]:
# пример

res = sel.fit_transform(X_tr, y)
display(res.shape)
res

In [ ]:
pipe_lasso =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', SVC(kernel='linear'))])

In [ ]:
cv_res4 = cross_validate(pipe_lasso, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res4

In [ ]:
cv_res4['test_score'].mean()

# PCA или метод главных компонент

Цель: создать k новых признаков (обычно k <= 5) из какого-либо количества старых признаков, так чтобы 
- каждый из новых признаков был линейной комбинацией старых

$z_i = u_1x_{1i} + ... + u_lx{li}$

- и дисперсия $z_i$, то есть новых признаков была максимальной (наиболее информативной)

С точки зрения линеной алгебры, процесс нахождения новых признаков из старых - это процесс проекции старых признаков на некоторую гиперплоскоть (линейное пространство). Как было показано на лекции, базисом этого пространства являются собственные вектора матрицы $X^TX$ - где Х - это центрированная матрица признаков

Тогда чтобы найти новые признаки (главные компоненты) нужно сначала
- найти собственные вектора V матрицы $X^TX$ (вектора должны быть приведены к длине 1)
- произвести матричное умножение Z = XV (то есть сделать проекцию матрицы X на линейное пространство с базисом V)

## Задание

a) Есть два признака, x1 = (1, 0, 0, 3), x2 = (3, 2, 0, 3). Найдите первую и вторую главные
компоненты.

б) Сколько дисперсии объясняется первой компонентой ?

Взято из задачника Б.Б. Демешева

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
# пример

pca = PCA(n_components = 15)

In [ ]:
res = pca.fit_transform(X_tr, y)
display(res.shape)
res

In [ ]:
# суммарная доля объясненной дисперсии исходных признаков

pca.explained_variance_ratio_.sum()

In [ ]:
# каждая следующая компонента менее информативна чем предыдущая
pca.explained_variance_ratio_

In [ ]:
n_components = 3

pipe_pca = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', PCA(n_components=n_components)), 
    ('model_', SVC(kernel='linear'))])

cv_res5 = cross_validate(pipe_pca, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

In [ ]:
cv_res5['test_score'].mean()